#4. Mini Banking System with Transaction History
Design a Mini Banking System that simulates basic banking operations.
The system should support account creation, deposits, withdrawals, and fund transfers
between accounts. Each transaction must be recorded and retrievable as a mini statement.
Business rules include minimum balance enforcement, daily withdrawal limits, and
rejection of invalid transactions. Data persistence during runtime and menu-based
navigation are mandatory.


In [14]:
MIN_BALANCE = 1000
DAILY_WITHDRAW_LIMIT = 10000

accounts = {}   # account_number -> account details


def create_account():
    acc_no = input("Enter Account Number: ").strip()
    if acc_no in accounts:
        print("Account already exists.")
        return

    name = input("Enter Account Holder Name: ").strip()

    try:
        balance = float(input("Enter Initial Balance: "))
        if balance < MIN_BALANCE:
            print("Minimum balance must be ₹1000.")
            return
    except ValueError:
        print("Invalid amount.")
        return

    accounts[acc_no] = {
        "name": name,
        "balance": balance,
        "withdraw_today": 0,
        "transactions": [("Account Created", balance)]
    }
    print("Account created successfully.")


def deposit():
    acc_no = input("Enter Account Number: ").strip()
    if acc_no not in accounts:
        print("Account not found.")
        return

    try:
        amount = float(input("Enter Deposit Amount: "))
        if amount <= 0:
            raise ValueError
    except ValueError:
        print("Invalid amount.")
        return

    accounts[acc_no]["balance"] += amount
    accounts[acc_no]["transactions"].append(("Deposit", amount))
    print("Deposit successful.")


def withdraw():
    acc_no = input("Enter Account Number: ").strip()
    if acc_no not in accounts:
        print("Account not found.")
        return

    try:
        amount = float(input("Enter Withdrawal Amount: "))
        if amount <= 0:
            raise ValueError
    except ValueError:
        print("Invalid amount.")
        return

    acc = accounts[acc_no]

    if acc["withdraw_today"] + amount > DAILY_WITHDRAW_LIMIT:
        print("Daily withdrawal limit exceeded.")
        return
    if acc["balance"] - amount < MIN_BALANCE:
        print("Minimum balance rule violated.")
        return

    acc["balance"] -= amount
    acc["withdraw_today"] += amount
    acc["transactions"].append(("Withdraw", amount))
    print("Withdrawal successful.")


def transfer():
    from_acc = input("Enter Sender Account Number: ").strip()
    to_acc = input("Enter Receiver Account Number: ").strip()

    if from_acc not in accounts or to_acc not in accounts:
        print("Invalid account number.")
        return
    if from_acc == to_acc:
        print("Cannot transfer to same account.")
        return

    try:
        amount = float(input("Enter Transfer Amount: "))
        if amount <= 0:
            raise ValueError
    except ValueError:
        print("Invalid amount.")
        return

    if accounts[from_acc]["balance"] - amount < MIN_BALANCE:
        print("Insufficient balance.")
        return

    accounts[from_acc]["balance"] -= amount
    accounts[to_acc]["balance"] += amount

    accounts[from_acc]["transactions"].append(("Transfer Sent", amount))
    accounts[to_acc]["transactions"].append(("Transfer Received", amount))

    print("Transfer successful.")


def mini_statement():
    acc_no = input("Enter Account Number: ").strip()
    if acc_no not in accounts:
        print("Account not found.")
        return

    print(f"\nMini Statement for {accounts[acc_no]['name']}")
    for txn in accounts[acc_no]["transactions"]:
        print(txn[0], "₹", txn[1])
    print("Current Balance: ₹", accounts[acc_no]["balance"])


def menu():
    while True:
        print("\n--- Mini Banking System ---")
        print("1. Create Account")
        print("2. Deposit")
        print("3. Withdraw")
        print("4. Transfer Funds")
        print("5. Mini Statement")
        print("6. Exit")

        choice = input("Enter choice: ").strip()

        if choice == "1":
            create_account()
        elif choice == "2":
            deposit()
        elif choice == "3":
            withdraw()
        elif choice == "4":
            transfer()
        elif choice == "5":
            mini_statement()
        elif choice == "6":
            print("Thank you for using the Banking System.")
            break
        else:
            print("Invalid choice.")


menu()



--- Mini Banking System ---
1. Create Account
2. Deposit
3. Withdraw
4. Transfer Funds
5. Mini Statement
6. Exit
Enter choice: 1
Enter Account Number: 235
Enter Account Holder Name: nisha
Enter Initial Balance: 2000
Account created successfully.

--- Mini Banking System ---
1. Create Account
2. Deposit
3. Withdraw
4. Transfer Funds
5. Mini Statement
6. Exit
Enter choice: 2
Enter Account Number: 235
Enter Deposit Amount: 1000
Deposit successful.

--- Mini Banking System ---
1. Create Account
2. Deposit
3. Withdraw
4. Transfer Funds
5. Mini Statement
6. Exit
Enter choice: 3
Enter Account Number: 235
Enter Withdrawal Amount: 500
Withdrawal successful.

--- Mini Banking System ---
1. Create Account
2. Deposit
3. Withdraw
4. Transfer Funds
5. Mini Statement
6. Exit
Enter choice: 4
Enter Sender Account Number: 235
Enter Receiver Account Number: 456
Invalid account number.

--- Mini Banking System ---
1. Create Account
2. Deposit
3. Withdraw
4. Transfer Funds
5. Mini Statement
6. Exit
Enter 

#1. Smart Library Circulation System
Design a console-based Smart Library Circulation System for a college department. The
system should manage books, students, and borrowing activity in an offline environment.
The application must be menu-driven and continuously run until the user chooses to exit.
Users should be able to add new books with details such as book ID, title, author, total
copies, and available copies. Students must be registered using a unique student ID and
name.
The system should allow issuing books to students while enforcing rules such as:
• A book can be issued only if copies are available.
• A student can issue only a limited number of books.
• The same book cannot be issued twice to the same student.
Returning a book should update availability and calculate overdue penalties based on the
number of days exceeded. The program should also support searching books using partial
string matching and displaying overdue reports.
Appropriate data structures like dictionaries, lists, tuples, and strings must be used. Input
validation and logical consistency are mandatory

In [15]:
MAX_BOOKS_PER_STUDENT = 3
BORROW_DAYS_LIMIT = 14
FINE_PER_DAY = 5

books = {}        # book_id -> details
students = {}     # student_id -> name
issued_books = {} # student_id -> list of (book_id, days)


def add_book():
    book_id = input("Enter Book ID: ").strip()
    if book_id in books:
        print("Book already exists.")
        return

    title = input("Enter Title: ").strip()
    author = input("Enter Author: ").strip()

    try:
        total = int(input("Enter Total Copies: "))
        if total <= 0:
            raise ValueError
    except ValueError:
        print("Invalid number of copies.")
        return

    books[book_id] = {
        "title": title,
        "author": author,
        "total": total,
        "available": total
    }
    print("Book added successfully.")


def register_student():
    student_id = input("Enter Student ID: ").strip()
    if student_id in students:
        print("Student already registered.")
        return

    name = input("Enter Student Name: ").strip()
    students[student_id] = name
    issued_books[student_id] = []
    print("Student registered successfully.")


def issue_book():
    student_id = input("Enter Student ID: ").strip()
    book_id = input("Enter Book ID: ").strip()

    if student_id not in students:
        print("Student not found.")
        return
    if book_id not in books:
        print("Book not found.")
        return
    if books[book_id]["available"] <= 0:
        print("No copies available.")
        return
    if len(issued_books[student_id]) >= MAX_BOOKS_PER_STUDENT:
        print("Book issue limit reached.")
        return
    if any(book_id == b[0] for b in issued_books[student_id]):
        print("Same book already issued to this student.")
        return

    issued_books[student_id].append((book_id, 0))
    books[book_id]["available"] -= 1
    print("Book issued successfully.")


def return_book():
    student_id = input("Enter Student ID: ").strip()
    book_id = input("Enter Book ID: ").strip()

    if student_id not in issued_books:
        print("Invalid student.")
        return

    for record in issued_books[student_id]:
        if record[0] == book_id:
            try:
                days = int(input("Enter number of days borrowed: "))
                if days < 0:
                    raise ValueError
            except ValueError:
                print("Invalid days.")
                return

            issued_books[student_id].remove(record)
            books[book_id]["available"] += 1

            overdue_days = max(0, days - BORROW_DAYS_LIMIT)
            fine = overdue_days * FINE_PER_DAY

            print("Book returned successfully.")
            print("Overdue Days:", overdue_days)
            print("Fine: ₹", fine)
            return

    print("This book was not issued to the student.")


def search_books():
    keyword = input("Enter search keyword: ").lower()
    found = False

    for book_id, details in books.items():
        if keyword in details["title"].lower() or keyword in details["author"].lower():
            print(book_id, details)
            found = True

    if not found:
        print("No matching books found.")


def overdue_report():
    print("\nOverdue Report:")
    for student_id, records in issued_books.items():
        for book_id, days in records:
            if days > BORROW_DAYS_LIMIT:
                overdue = days - BORROW_DAYS_LIMIT
                fine = overdue * FINE_PER_DAY
                print(
                    "Student:", students[student_id],
                    "| Book:", books[book_id]["title"],
                    "| Fine: ₹", fine
                )


def menu():
    while True:
        print("\n--- Smart Library Circulation System ---")
        print("1. Add Book")
        print("2. Register Student")
        print("3. Issue Book")
        print("4. Return Book")
        print("5. Search Book")
        print("6. Overdue Report")
        print("7. Exit")

        choice = input("Enter choice: ").strip()

        if choice == "1":
            add_book()
        elif choice == "2":
            register_student()
        elif choice == "3":
            issue_book()
        elif choice == "4":
            return_book()
        elif choice == "5":
            search_books()
        elif choice == "6":
            overdue_report()
        elif choice == "7":
            print("Exiting system.")
            break
        else:
            print("Invalid choice.")


menu()



--- Smart Library Circulation System ---
1. Add Book
2. Register Student
3. Issue Book
4. Return Book
5. Search Book
6. Overdue Report
7. Exit
Enter choice: 1
Enter Book ID: 15
Enter Title: lokaha
Enter Author: sreenivasan
Enter Total Copies: 8
Book added successfully.

--- Smart Library Circulation System ---
1. Add Book
2. Register Student
3. Issue Book
4. Return Book
5. Search Book
6. Overdue Report
7. Exit
Enter choice: 2
Enter Student ID: 234
Enter Student Name: karthi
Student registered successfully.

--- Smart Library Circulation System ---
1. Add Book
2. Register Student
3. Issue Book
4. Return Book
5. Search Book
6. Overdue Report
7. Exit
Enter choice: 3
Enter Student ID: 234
Enter Book ID: 15
Book issued successfully.

--- Smart Library Circulation System ---
1. Add Book
2. Register Student
3. Issue Book
4. Return Book
5. Search Book
6. Overdue Report
7. Exit
Enter choice: 4
Enter Student ID: 234
Enter Book ID: 15
Enter number of days borrowed: 5
Book returned successfully.
